# Chapter 26

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# install Pint if necessary

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# download modsim.py if necessary

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# import functions from modsim

from modsim import *

Case studies!

## Bungee jumping

Suppose you want to set the world record for the highest "bungee dunk",
which is a stunt in which a bungee jumper dunks a cookie in a cup of tea
at the lowest point of a jump. An example is shown in this video:
<http://modsimpy.com/dunk>.

Since the record is 70 m, let's design a jump for 80 m. We'll start with
the following modeling assumptions:

-   Initially the bungee cord hangs from a crane with the attachment
    point 80 m above a cup of tea.

-   Until the cord is fully extended, it applies no force to the jumper.
    It turns out this might not be a good assumption; we will revisit
    it.

-   After the cord is fully extended, it obeys Hooke's Law; that is, it
    applies a force to the jumper proportional to the extension of the
    cord beyond its resting length. See <http://modsimpy.com/hooke>.

-   The mass of the jumper is 75 kg.

-   The jumper is subject to drag force so that their terminal velocity
    is 60 m/s.

Our objective is to choose the length of the cord, `L`, and its spring
constant, `k`, so that the jumper falls all the way to the tea cup, but
no farther!

In the repository for this book, you will find a notebook,
`bungee.ipynb`, which contains starter code and exercises for this case
study.

## Bungee dunk revisited

In the previous case study, we assume that the cord applies no force to
the jumper until it is stretched. It is tempting to say that the cord
has no effect because it falls along with the jumper, but that intuition
is incorrect. As the cord falls, it transfers energy to the jumper.

At <http://modsimpy.com/bungee> you'll find a paper[^1] that explains
this phenomenon and derives the acceleration of the jumper, $a$, as a
function of position, $y$, and velocity, $v$:
$$a = g + \frac{\mu v^2/2}{\mu(L+y) + 2L}$$ where $g$ is acceleration
due to gravity, $L$ is the length of the cord, and $\mu$ is the ratio of
the mass of the cord, $m$, and the mass of the jumper, $M$.

If you don't believe that their model is correct, this video might
convince you: <http://modsimpy.com/drop>.

In the repository for this book, you will find a notebook,
`bungee2.ipynb`, which contains starter code and exercises for this case
study. How does the behavior of the system change as we vary the mass of
the cord? When the mass of the cord equals the mass of the jumper, what
is the net effect on the lowest point in the jump?

## Spider-Man

In this case study we'll develop a model of Spider-Man swinging from a
springy cable of webbing attached to the top of the Empire State
Building. Initially, Spider-Man is at the top of a nearby building, as
shown in Figure [\[spiderman\]](#spiderman){reference-type="ref"
reference="spiderman"}.

![Diagram of the initial state for the Spider-Man case
study.](figs/spiderman.pdf){height="2.8in"}

The origin, `O`, is at the base of the Empire State Building. The vector
`H` represents the position where the webbing is attached to the
building, relative to `O`. The vector `P` is the position of Spider-Man
relative to `O`. And `L` is the vector from the attachment point to
Spider-Man.

By following the arrows from `O`, along `H`, and along `L`, we can see
that

H + L = P

So we can compute `L` like this:

L = P - H

The goals of this case study are:

1.  Implement a model of this scenario to predict Spider-Man's
    trajectory.

2.  Choose the right time for Spider-Man to let go of the webbing in
    order to maximize the distance he travels before landing.

3.  Choose the best angle for Spider-Man to jump off the building, and
    let go of the webbing, to maximize range.

We'll use the following parameters:

1.  According to the Spider-Man Wiki[^2], Spider-Man weighs 76 kg.

2.  Let's assume his terminal velocity is 60 m/s.

3.  The length of the web is 100 m.

4.  The initial angle of the web is 45 ° to the left of straight down.

5.  The spring constant of the web is 40 N/m when the cord is stretched,
    and 0 when it's compressed.

In the repository for this book, you will find a notebook,
`spiderman.ipynb`, which contains starter code. Read through the
notebook and run the code. It uses `minimize`, which is a SciPy function
that can search for an optimal set of parameters (as contrasted with
`minimize_scalar`, which can only search along a single axis).

## Kittens

Let's simulate a kitten unrolling toilet paper. As reference material,
see this video: <http://modsimpy.com/kitten>.

The interactions of the kitten and the paper roll are complex. To keep
things simple, let's assume that the kitten pulls down on the free end
of the roll with constant force. Also, we will neglect the friction
between the roll and the axle.

![Diagram of a roll of toilet paper, showing a force, lever arm, and the
resulting torque.](figs/kitten.pdf){height="2.5in"}

Figure [\[kitten\]](#kitten){reference-type="ref" reference="kitten"}
shows the paper roll with $r$, $F$, and $\tau$. As a vector quantity,
the direction of $\tau$ is into the page, but we only care about its
magnitude for now.

Here's the `Params` object with the parameters we'll need:

In [ ]:
params = Params(Rmin = 0.02 * m,
                Rmax = 0.055 * m,
                Mcore = 15e-3 * kg,
                Mroll = 215e-3 * kg,
                L = 47 * m,
                tension = 2e-4 * N,
                t_end = 180 * s)

As before, `Rmin` is the minimum radius and `Rmax` is the maximum. `L`
is the length of the paper. `Mcore` is the mass of the cardboard tube at
the center of the roll; `Mroll` is the mass of the paper. `tension` is
the force applied by the kitten, in N. I chose a value that yields
plausible results.

At <http://modsimpy.com/moment> you can find moments of inertia for
simple geometric shapes. I'll model the cardboard tube at the center of
the roll as a "thin cylindrical shell\", and the paper roll as a
"thick-walled cylindrical tube with open ends\".

The moment of inertia for a thin shell is just $m r^2$, where $m$ is the
mass and $r$ is the radius of the shell.

For a thick-walled tube the moment of inertia is
$$I = \frac{\pi \rho h}{2} (r_2^4 - r_1^4)$$ where $\rho$ is the density
of the material, $h$ is the height of the tube, $r_2$ is the outer
diameter, and $r_1$ is the inner diameter.

Since the outer diameter changes as the kitten unrolls the paper, we
have to compute the moment of inertia, at each point in time, as a
function of the current radius, `r`. Here's the function that does it:

In [ ]:
def moment_of_inertia(r, system):
    Mcore, Rmin = system.Mcore, system.Rmin
    rho_h = system.rho_h
    
    Icore = Mcore * Rmin**2   
    Iroll = pi * rho_h / 2 * (r**4 - Rmin**4)
    return Icore + Iroll

`rho_h` is the product of density and height, $\rho h$, which is the
mass per area. `rho_h` is computed in `make_system`:

In [ ]:
def make_system(params):
    L, Rmax, Rmin = params.L, params.Rmax, params.Rmin
    Mroll = params.Mroll
    
    init = State(theta = 0 * radian,
                 omega = 0 * radian/s,
                 y = L)
    
    area = pi * (Rmax**2 - Rmin**2)
    rho_h = Mroll / area
    k = (Rmax**2 - Rmin**2) / 2 / L / radian    
    
    return System(params, init=init, area=area, 
                  rho_h=rho_h, k=k)

`make_system` also computes `k` using
Equation [\[eqn4\]](#eqn4){reference-type="ref" reference="eqn4"}.

In the repository for this book, you will find a notebook,
`kitten.ipynb`, which contains starter code for this case study. Use it
to implement this model and check whether the results seem plausible.

## Simulating a yo-yo

Suppose you are holding a yo-yo with a length of string wound around its
axle, and you drop it while holding the end of the string stationary. As
gravity accelerates the yo-yo downward, tension in the string exerts a
force upward. Since this force acts on a point offset from the center of
mass, it exerts a torque that causes the yo-yo to spin.

![Diagram of a yo-yo showing forces due to gravity and tension in the
string, the lever arm of tension, and the resulting
torque.](figs/yoyo.pdf){height="2.5in"}

Figure [\[yoyo\]](#yoyo){reference-type="ref" reference="yoyo"} is a
diagram of the forces on the yo-yo and the resulting torque. The outer
shaded area shows the body of the yo-yo. The inner shaded area shows the
rolled up string, the radius of which changes as the yo-yo unrolls.

In this model, we can't figure out the linear and angular acceleration
independently; we have to solve a system of equations: $$\begin{aligned}
\sum F &= m a \\
\sum \tau &= I \alpha\end{aligned}$$ where the summations indicate that
we are adding up forces and torques.

As in the previous examples, linear and angular velocity are related
because of the way the string unrolls:
$$\frac{dy}{dt} = -r \frac{d \theta}{dt}$$ In this example, the linear
and angular accelerations have opposite sign. As the yo-yo rotates
counter-clockwise, $\theta$ increases and $y$, which is the length of
the rolled part of the string, decreases.

Taking the derivative of both sides yields a similar relationship
between linear and angular acceleration:
$$\frac{d^2 y}{dt^2} = -r \frac{d^2 \theta}{dt^2}$$ Which we can write
more concisely: $$a = -r \alpha$$ This relationship is not a general law
of nature; it is specific to scenarios like this where one object rolls
along another without stretching or slipping.

Because of the way we've set up the problem, $y$ actually has two
meanings: it represents the length of the rolled string and the height
of the yo-yo, which decreases as the yo-yo falls. Similarly, $a$
represents acceleration in the length of the rolled string and the
height of the yo-yo.

We can compute the acceleration of the yo-yo by adding up the linear
forces: $$\sum F = T - mg = ma$$ Where $T$ is positive because the
tension force points up, and $mg$ is negative because gravity points
down.

Because gravity acts on the center of mass, it creates no torque, so the
only torque is due to tension: $$\sum \tau = T r = I \alpha$$ Positive
(upward) tension yields positive (counter-clockwise) angular
acceleration.

Now we have three equations in three unknowns, $T$, $a$, and $\alpha$,
with $I$, $m$, $g$, and $r$ as known quantities. It is simple enough to
solve these equations by hand, but we can also get SymPy to do it for
us:

In [ ]:
T, a, alpha, I, m, g, r = symbols('T a alpha I m g r')
eq1 = Eq(a, -r * alpha)
eq2 = Eq(T - m*g, m * a)
eq3 = Eq(T * r, I * alpha)
soln = solve([eq1, eq2, eq3], [T, a, alpha])

The results are $$\begin{aligned}
T      &= m g I / I^*   \\
a      &= -m g r^2 / I^* \\
\alpha &= m g r / I^*    \\\end{aligned}$$ where $I^*$ is the augmented
moment of inertia, $I + m r^2$. To simulate the system, we don't really
need $T$; we can plug $a$ and $\alpha$ directly into the slope function.

In the repository for this book, you will find a notebook, `yoyo.ipynb`,
which contains the derivation of these equations and starter code for
this case study. Use it to implement and test this model.

[^1]: Heck, Uylings, and Kędzierska, "Understanding the physics of
    bungee jumping\", Physics Education, Volume 45, Number 1, 2010.

[^2]: <http://modsimpy.com/spider>